>Accuracy

In [4]:
from sklearn.base import BaseEstimator
import numpy as np

class MyDummyClassifier(BaseEstimator):
    def fit(self, X, Y=None):
        pass

    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range (X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
            
        return pred

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

titanic_df = pd.read_csv(r'.\titanic\train.csv')
encoder = LabelEncoder()
encoder.fit(titanic_df['Sex'])
titanic_df['Sex'] = encoder.transform(titanic_df['Sex'])

target = titanic_df['Survived']
data = titanic_df.drop('Survived', axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.2, random_state=0)

my_clf = MyDummyClassifier()
my_clf.fit(X_train, Y_train)
predictions = my_clf.predict(X_test)
print('Accuracy of Dummy Classifier:', accuracy_score(Y_test, predictions))

Accuracy of Dummy Classifier: 0.7877094972067039


In [33]:
from sklearn.datasets import load_digits


class MyFakeClassifier(BaseEstimator):
    def fit(self, X, Y=None):
        pass

    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)


digits = load_digits()

y = (digits.target == 7).astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(
    digits.data, y, random_state=11)


In [34]:
print('# of data:', digits.target.shape[0])
trues = y[y==True]
print('# of 7 in data:', trues.shape[0])
print('Ratio of 7 in data:',trues.shape[0]/digits.target.shape[0])

# of data: 1797
# of 7 in data: 179
Ratio of 7 in data: 0.09961046188091263


In [35]:
print('Size of test label set:', Y_test.shape)
print('Dist. of test label set:\n', pd.Series(Y_test).value_counts(), sep='')

fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, Y_train)
fakepred = fakeclf.predict(X_test)
print('Accuracy:', accuracy_score(Y_test, fakepred))

Size of test label set: (450,)
Dist. of test label set:
0    405
1     45
dtype: int64
Accuracy: 0.9


>Confusion Matrix

In [36]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, fakepred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

>Precision and Recall

In [42]:
#Precision = TP/(FP+TP)
#Recall = TP/(FN+TP)
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.metrics import recall_score, precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('Confusion Matrix:\n', confusion, sep='')
    print('Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}'.format(
        accuracy, precision, recall))
    

In [43]:
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

def label_encoding(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder.fit(df[feature])
        df[feature] = encoder.transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = label_encoding(df)
    return df

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

titanic_df = pd.read_csv(r'.\titanic\train.csv')

target = titanic_df['Survived']
data = titanic_df.drop('Survived', axis=1)
data = transform_features(data)

X_train, X_test, Y_train, Y_test = train_test_split(
    data, target, test_size=0.2, random_state=11)

LogReg = LogisticRegression()
LogReg.fit(X_train, Y_train)
LG_pred = LogReg.predict(X_test)
get_clf_eval(LG_pred, Y_test)


Confusion Matrix:
[[104  13]
 [ 14  48]]
Accuracy: 0.8492, Precision: 0.7869, Recall: 0.7742


c:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


>Trade-off between Precision & Recall